# Multi-material NEB Pipeline (Restartable, with time-logged steps)

This notebook builds a **restartable, multi-material NEB workflow** for
surface diffusion of simple metal anodes (Li, Na, K, Mg, Al).

Key features:

- All slabs are built **directly from ASE** (`surface()`), no external files.
- Uses **`ase.mep.NEB`**.
- Each important step (initial relaxation, final relaxation, NEB) is
  tracked in a small `status.json` file so you can **restart after kernel
  shutdown** without redoing finished work.
- Relaxations and NEB run step-by-step with **text logging**, including
  **elapsed wall-clock time per step**, so you can see progress.


## 1. Environment and dependencies

Requirements:

- Python environment with **ASE** and **GPAW** working.
- No extra Python packages beyond the standard library.

On a cluster, you might need (example, adjust to your system):

```bash
module load gpaw ase
```

Then start Jupyter and run this notebook.


In [4]:
# 2. Imports
from ase.build import surface
from ase.io import write, read
from ase import Atom
from ase.optimize import BFGS, FIRE
from ase.mep import NEB
from gpaw import GPAW, PW
from ase.parallel import world
from ase.constraints import FixAtoms

import numpy as np
import os
import json
import time

print("Imports OK")


Imports OK


## 3. Global parameters

Edit this cell to change:

- Which materials and surfaces to study.
- Slab size, vacuum, and adatom height.
- GPAW and NEB convergence settings.

All slabs are generated from scratch; no external structure files are needed.


In [5]:
# 3. Global parameters

materials = [
    ("Li", (1, 1, 0), "Li bcc (110)"),
    ("Li", (1, 1, 1), "Li bcc (111)"),
    ("Li", (1, 0, 0), "Li bcc (100)"),
]

layers = 4
supercell = (3, 3, 1)
vacuum = 10.0
adatom_height = 2.0

relax_slab = False
relax_initial_final = True

kp = 2
PWc = 200
xc = "PBE"
slab_fmax = 0.05
initial_final_fmax = 0.05
neb_fmax = 0.10

n_middle_images = 5
neb_max_steps = 300

output_dir = "crystal_structure_neb_results"
os.makedirs(output_dir, exist_ok=True)
status_path = os.path.join(output_dir, "status.json")

print(f"Output directory: {output_dir}")
print(f"Status file: {status_path}")


Output directory: crystal_structure_neb_results
Status file: crystal_structure_neb_results/status.json


## 4. Helper functions

Defines:
- Slab building and adatom placement.
- GPAW calculator factory.
- Status load/save.
- Restart-aware relaxation with time-logged steps.


In [6]:
def make_label(symbol, description, indices):
    desc_part = description if description else f"hkl-{indices[0]}{indices[1]}{indices[2]}"
    raw = f"{symbol}-{desc_part}"
    safe = [c if (c.isalnum() or c in "-_") else "-" for c in raw]
    return "".join(safe).lower()


def build_slab(symbol, indices):
    slab = surface(symbol, indices, layers=layers, vacuum=vacuum)
    slab = slab.repeat(supercell)
    return slab


def choose_two_top_sites(slab, initial_dz=0.5):
    positions = slab.get_positions()
    z = positions[:, 2]
    z_max = z.max()

    for dz in [initial_dz, 1.0, 2.0]:
        top_indices = np.where(z > z_max - dz)[0]
        if len(top_indices) >= 2:
            break
    else:
        raise RuntimeError(f"Not enough top-layer atoms found (max z = {z_max:.3f}).")

    i1 = top_indices[0]
    p1 = positions[i1]

    best_j = None
    best_d = 1e9
    for j in top_indices[1:]:
        dp = positions[j] - p1
        d_xy = np.linalg.norm(dp[:2])
        if 1e-5 < d_xy < best_d:
            best_d = d_xy
            best_j = j

    if best_j is None:
        raise RuntimeError("Could not find a suitable neighbouring top-layer site.")

    p2 = positions[best_j]
    return p1, p2, z_max


def apply_bottom_fix(slab, fraction=0.5):
    """Freeze the bottom `fraction` of the slab to emulate bulk anchoring."""
    z = slab.get_positions()[:, 2]
    z_min, z_max = z.min(), z.max()
    z_cut = z_min + fraction * (z_max - z_min)
    mask = z < z_cut
    slab.set_constraint(FixAtoms(mask=mask))
    return slab


def make_adatom_state(base_slab, symbol, site_pos, z_max):
    """Copy the slab, fix the bottom, and place an adatom above `site_pos`."""
    slab = base_slab.copy()
    slab = apply_bottom_fix(slab, fraction=0.5)
    adatom = Atom(symbol, [site_pos[0], site_pos[1], z_max + adatom_height])
    slab.append(adatom)
    return slab


def make_gpaw(label):
    """Factory for a GPAW calculator (plane-wave mode) with shared settings."""
    return GPAW(
        mode=PW(PWc),
        xc=xc,
        kpts=(kp, kp, 1),
        convergence={"eigenstates": 5e-6, "density": 1e-4},
        txt=os.path.join(output_dir, f"{label}.txt"),
    )


def load_status():
    if not os.path.exists(status_path):
        return {}
    try:
        with open(status_path, "r") as f:
            return json.load(f)
    except Exception:
        return {}


def save_status(status):
    with open(status_path, "w") as f:
        json.dump(status, f, indent=2)


def relax_structure(atoms, label, fmax, allow_restart=True):
    """Relax `atoms` with BFGS, optionally restarting from the last trajectory."""
    traj_path = os.path.join(output_dir, f"{label}.traj")

    if allow_restart and os.path.exists(traj_path):
        if world.rank == 0:
            print(f"[{label}] Restarting from existing trajectory: {traj_path}")
        try:
            atoms = read(traj_path, index=-1)
        except Exception as exc:
            if world.rank == 0:
                print(f"[{label}] Failed to read traj ({exc}); starting fresh.")

    atoms.calc = make_gpaw(label)
    dyn = BFGS(
        atoms,
        logfile=os.path.join(output_dir, f"{label}.log"),
        trajectory=traj_path,
    )

    step = 0
    t0 = time.time()
    if world.rank == 0:
        print(f"[{label}] Starting relaxation (target fmax = {fmax:.3f} eV/Å)")

    for _ in dyn.irun(fmax=fmax):
        step += 1
        if world.rank == 0:
            elapsed = time.time() - t0
            forces = atoms.get_forces()
            maxf = float(np.abs(forces).max())
            print(f"[{label}] step {step:4d}, max |F| = {maxf:.3f} eV/Å, elapsed = {elapsed:7.1f} s")

    if world.rank == 0:
        total = time.time() - t0
        print(f"[{label}] Relaxation finished in {step} steps, total time = {total:.1f} s.")
    return atoms


print("Helper functions defined.")


Helper functions defined.


## 5. NEB driver (restart-aware, per-step timing)

Runs NEB for a single material, using `status.json` to avoid redoing work,
        
and logs **each NEB optimization step** with elapsed time.


In [4]:
def run_neb_for_material(symbol, indices, description=""):
    """Run the NEB workflow for one material/structure combo, restart-aware."""
    label = make_label(symbol, description, indices)
    status = load_status()
    mat_status = status.get(label, {
        "initial_relaxed": False,
        "final_relaxed": False,
        "neb_done": False,
    })

    if world.rank == 0:
        print("\n" + "=" * 60)
        header = description if description else str(indices)
        print(f"Running NEB for {symbol} ({header}) -> label: {label}")
        print("Current status:", mat_status)
        print("=" * 60)

    slab = build_slab(symbol, indices)

    if world.rank == 0:
        write(os.path.join(output_dir, f"{label}-slab.xyz"), slab)

    p1, p2, z_max = choose_two_top_sites(slab)

    initial = make_adatom_state(slab, symbol, p1, z_max)
    final = make_adatom_state(slab, symbol, p2, z_max)

    if relax_initial_final:
        initial_path = os.path.join(output_dir, f"{label}-initial.traj")
        final_path = os.path.join(output_dir, f"{label}-final.traj")

        if not mat_status.get("initial_relaxed", False):
            if world.rank == 0:
                print(f"{label}: relaxing initial adatom configuration...")
            initial = relax_structure(initial, f"{label}-initial", initial_final_fmax, allow_restart=True)
            mat_status["initial_relaxed"] = True
            status[label] = mat_status
            save_status(status)
        else:
            if world.rank == 0:
                print(f"{label}: initial state already relaxed (skipping).")
            initial = read(initial_path, index=-1)

        if not mat_status.get("final_relaxed", False):
            if world.rank == 0:
                print(f"{label}: relaxing final adatom configuration...")
            final = relax_structure(final, f"{label}-final", initial_final_fmax, allow_restart=True)
            mat_status["final_relaxed"] = True
            status[label] = mat_status
            save_status(status)
        else:
            if world.rank == 0:
                print(f"{label}: final state already relaxed (skipping).")
            final = read(final_path, index=-1)

    if world.rank == 0:
        write(os.path.join(output_dir, f"{label}-initial.xyz"), initial)
        write(os.path.join(output_dir, f"{label}-final.xyz"), final)

    if mat_status.get("neb_done", False):
        if world.rank == 0:
            print(f"{label}: NEB already marked as done. Skipping NEB run.")
        energies_file = os.path.join(output_dir, f"{label}-NEB-energies.dat")
        if os.path.exists(energies_file):
            data = np.loadtxt(energies_file, comments="#")
            energies = data[:, 1]
            ref = min(energies[0], energies[-1])
            barrier = energies.max() - ref
            return barrier
        else:
            if world.rank == 0:
                print(f"{label}: NEB marked done but energies file missing. Rerunning NEB.")
            mat_status["neb_done"] = False
            status[label] = mat_status
            save_status(status)

    images = [initial]
    images += [initial.copy() for _ in range(n_middle_images)]
    images += [final]

    neb = NEB(images, climb=True)
    neb.interpolate()

    for i, img in enumerate(images):
        img.calc = make_gpaw(f"{label}-NEB-img{i}")

    if world.rank == 0:
        print(f"{label}: starting NEB optimization with {len(images)} images "
              f"(target fmax = {neb_fmax:.3f} eV/Å)")

    opt = FIRE(
        neb,
        logfile=os.path.join(output_dir, f"{label}-NEB.log"),
        trajectory=os.path.join(output_dir, f"{label}-NEB.traj"),
    )

    step = 0
    t0 = time.time()
    for _ in opt.irun(fmax=neb_fmax, steps=neb_max_steps):
        step += 1
        if world.rank == 0:
            elapsed = time.time() - t0
            try:
                neb_forces = neb.get_forces()
                maxf = float(np.abs(neb_forces).max())
            except Exception:
                maxf = float("nan")
            print(f"[{label} NEB] step {step:4d}, NEB max |F| = {maxf:.3f} eV/Å, elapsed = {elapsed:7.1f} s")

    if world.rank == 0:
        total = time.time() - t0
        print(f"[{label} NEB] optimization finished after {step} steps, total time = {total:.1f} s.")

    energies = np.array([img.get_potential_energy() for img in images])
    ref = min(energies[0], energies[-1])
    barrier = energies.max() - ref

    if world.rank == 0:
        print(f"{label}: diffusion barrier ≈ {barrier:.3f} eV")
        energies_file = os.path.join(output_dir, f"{label}-NEB-energies.dat")
        with open(energies_file, "w") as f:
            f.write("# image   energy_eV\n")
            for i, E in enumerate(energies):
                f.write(f"{i:3d}  {E:15.8f}\n")

    mat_status["neb_done"] = True
    status[label] = mat_status
    save_status(status)

    return barrier

print("NEB driver function defined (with time-logged steps).")

NEB driver function defined (with time-logged steps).


## 6. Run NEB for all materials

You can re-run this cell multiple times. It uses `status.json` and the
relax/NEB outputs to skip work that is already done.


In [5]:
summary_path = os.path.join(output_dir, "diffusion_barriers_summary.dat")

if world.rank == 0:
    sf = open(summary_path, "w")
    sf.write("# label                 barrier_eV\n")
else:
    sf = None

for symbol, indices, desc in materials:
    label = make_label(symbol, desc, indices)
    try:
        barrier = run_neb_for_material(symbol, indices, description=desc)
        if world.rank == 0:
            sf.write(f"{label:24s}  {barrier:10.4f}\n")
    except Exception as e:
        if world.rank == 0:
            print(f"{label}: FAILED – {e}")
            sf.write(f"{label:24s}  FAILED\n")

if sf is not None:
    sf.close()
    if world.rank == 0:
        print("\nAll done. Summary written to:", summary_path)



Running NEB for Li (Li bcc (110)) -> label: li-li-bcc--110-
Current status: {'initial_relaxed': True, 'final_relaxed': True, 'neb_done': False}
li-li-bcc--110-: initial state already relaxed (skipping).
li-li-bcc--110-: final state already relaxed (skipping).
li-li-bcc--110-: starting NEB optimization with 7 images (target fmax = 0.100 eV/Å)
[li-li-bcc--110- NEB] step    1, NEB max |F| = 0.303 eV/Å, elapsed =  1086.8 s
[li-li-bcc--110- NEB] step    1, NEB max |F| = 0.303 eV/Å, elapsed =  1086.8 s
[li-li-bcc--110- NEB] step    2, NEB max |F| = 0.292 eV/Å, elapsed =  1284.1 s
[li-li-bcc--110- NEB] step    2, NEB max |F| = 0.292 eV/Å, elapsed =  1284.1 s
[li-li-bcc--110- NEB] step    3, NEB max |F| = 0.270 eV/Å, elapsed =  1494.1 s
[li-li-bcc--110- NEB] step    3, NEB max |F| = 0.270 eV/Å, elapsed =  1494.1 s
[li-li-bcc--110- NEB] step    4, NEB max |F| = 0.239 eV/Å, elapsed =  1732.8 s
[li-li-bcc--110- NEB] step    4, NEB max |F| = 0.239 eV/Å, elapsed =  1732.8 s
[li-li-bcc--110- NEB] s

## 7. Visualization with NGLView

Visualize the initial/final structures and NEB trajectory animation using nglview.

In [2]:
import nglview as nv
from ase.io import read, Trajectory
import ipywidgets as widgets
from IPython.display import display

def show_structure(atoms, title="Structure", size=(600, 600)):
    """Display a single ASE Atoms object with nglview."""
    view = nv.show_ase(atoms)
    view._set_size(f"{size[0]}px", f"{size[1]}px")
    view.clear_representations()
    view.add_representation('spacefill', radius=0.5)
    view.add_representation('unitcell')
    view.parameters = dict(backgroundColor='white')
    view.camera = 'orthographic'
    print(title)
    return view

def show_neb_animation(traj_path, title="NEB Animation", size=(800, 800)):
    """Display NEB trajectory as an animation."""
    try:
        traj = read(traj_path, index=':')
    except Exception as e:
        print(f"Could not load trajectory: {e}")
        return None
    
    view = nv.show_asetraj(traj)
    view.clear_representations()
    view.add_representation('spacefill', radius=0.5)
    view.add_representation('unitcell')
    view.parameters = dict(backgroundColor='white')
    view.camera = 'orthographic'
    print(f"{title} ({len(traj)} frames)")
    return view

def show_initial_final_comparison(initial_path, final_path, title_prefix=""):
    """Display initial and final structures side by side."""
    initial = read(initial_path)
    final = read(final_path)
    
    print(f"{title_prefix} Initial vs Final Structures")
    print("-" * 40)
    
    # Initial structure
    view1 = show_structure(initial, f"{title_prefix} Initial", size=(500, 500))
    display(view1)
    
    # Final structure
    view2 = show_structure(final, f"{title_prefix} Final", size=(500, 500))
    display(view2)
    
    return view1, view2


print("Visualization functions defined.")

Visualization functions defined.


### 7.1 Visualize all materials: Initial and Final structures

In [7]:
# Visualize initial and final structures for each material
for symbol, indices, desc in materials:
    label = make_label(symbol, desc, indices)
    initial_path = os.path.join(output_dir, f"{label}-initial.xyz")
    final_path = os.path.join(output_dir, f"{label}-final.xyz")
    
    if os.path.exists(initial_path) and os.path.exists(final_path):
        print(f"\n{'='*60}")
        print(f"{desc} - Initial Structure")
        print(f"{'='*60}")
        initial_atoms = read(initial_path)
        view_init = show_structure(initial_atoms, f"{desc} Initial", size=(600, 600))
        display(view_init)
        
        print(f"\n{desc} - Final Structure")
        final_atoms = read(final_path)
        view_final = show_structure(final_atoms, f"{desc} Final", size=(600, 600))
        display(view_final)
    else:
        print(f"{label}: Structure files not found. Run the NEB calculation first.")


Li bcc (110) - Initial Structure
Li bcc (110) Initial


NGLWidget()


Li bcc (110) - Final Structure
Li bcc (110) Final


NGLWidget()


Li bcc (111) - Initial Structure
Li bcc (111) Initial


...

NGLWidget()


Li bcc (100) - Initial Structure
Li bcc (100) Initial


NGLWidget()


Li bcc (100) - Final Structure
Li bcc (100) Final


NGLWidget()

### 7.2 NEB Trajectory Animations

Animate the diffusion path for each material showing all NEB images.

In [ ]:
# Animate NEB trajectory for each material
for symbol, indices, desc in materials:
    label = make_label(symbol, desc, indices)
    neb_traj_path = os.path.join(output_dir, f"{label}-NEB.traj")
    
    if os.path.exists(neb_traj_path):
        print(f"\n{'='*60}")
        print(f"{desc} - NEB Diffusion Animation")
        print(f"{'='*60}")
        view = show_neb_animation(neb_traj_path, f"{desc} NEB Trajectory", size=(800, 800))
        if view:
            display(view)
    else:
        print(f"{label}: NEB trajectory not found. Run the NEB calculation first.")

### 7.3 Interactive NEB Image Viewer

Select a material and browse through individual NEB images with an interactive slider.

In [ ]:
# Interactive viewer for individual NEB images
def create_neb_image_viewer(label, desc, output_dir):
    """Create an interactive viewer to browse NEB images."""
    neb_traj_path = os.path.join(output_dir, f"{label}-NEB.traj")
    
    if not os.path.exists(neb_traj_path):
        print(f"{label}: NEB trajectory not found.")
        return None
    
    try:
        traj = read(neb_traj_path, index=':')
    except Exception as e:
        print(f"Error loading trajectory: {e}")
        return None
    
    # Create slider widget
    image_slider = widgets.IntSlider(
        value=0,
        min=0,
        max=len(traj)-1,
        step=1,
        description=f'Image:',
        continuous_update=False,
        style={'description_width': 'initial'}
    )
    
    output = widgets.Output()
    
    def update_view(change):
        with output:
            output.clear_output(wait=True)
            idx = change['new']
            atoms = traj[idx]
            print(f"Showing NEB image {idx}/{len(traj)-1}")
            view = show_structure(atoms, f"{desc} NEB Image {idx}", size=(600, 600))
            display(view)
    
    image_slider.observe(update_view, names='value')
    
    # Initial display
    with output:
        atoms = traj[0]
        print(f"Showing NEB image 0/{len(traj)-1}")
        view = show_structure(atoms, f"{desc} NEB Image 0", size=(600, 600))
        display(view)
    
    display(widgets.VBox([image_slider, output]))
    return image_slider

# Create viewers for each material
for symbol, indices, desc in materials:
    label = make_label(symbol, desc, indices)
    print(f"\n{'='*60}")
    print(f"{desc} - Interactive NEB Image Browser")
    print(f"{'='*60}")
    create_neb_image_viewer(label, desc, output_dir)